<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 40px; margin-top: 0;">
    <div style="flex: 0 0 auto; margin-left: 0; margin-bottom: 0; margin-top: 0;">
        <img src="./pics/UCSD Logo.png" alt="UCSD Logo" style="width: 179px; margin-bottom: 0px; margin-top: 20px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/silvxlabs.png" alt="SilvX Labs Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/sdsclogo-plusname-horiz-red.jpg" alt="San Diego Supercomputer Center Logo" width="300"/>
    </div>
</div>

<h1 style="text-align: center; font-size: 48px; margin-top: 0;">6NRP Demo Data Challenge</h1>

### Acknowledgment

This Jupyter Notebook was authored by Anthony Marcozzi from the New Mexico Consortium. The content, including the code, analysis, and visualizations, reflects the author's expertise and effort in creating an accessible and insightful resource. Proper credit is appreciated if this notebook is shared or adapted for further use.

# Introduction

Welcome to the 6NRP Demo Data Challenge! In this first notebook we will inroduce the problem at the center of the data challenge, and introduce some of the data sources that we'll use to solve the problem.

## Independence Lake - California
[Independence Lake Preserve](https://www.nature.org/en-us/get-involved/how-to-help/places-we-protect/independence-lake/) is approximately 3,000 acres located on the border between California and Nevada (25 miles east of Reno, NV) owned by The Nature Conservancy (TNC). In this area, there are a mix of conifer species including Jeffrey pine, ponderosa pine, lodgepole pine, along with white and red fir. Jeffrey, and lodgepole pine are dominant cover types and the dominant shrubs in the area are ceanothus, manzanita and huckleberry oak and there are some grassy meadows. The vegetation of Independence Lake Preserve has been shaped by historic logging and not many wildfires. At this site, TNC has established goals of promoting and enhancing biodiversity and wildfire resiliency, protecting and improving watershed function and wildlife habitat, as well as continuing the recreational and educational missions of TNC. Through the reintroduction of fire as a disturbance process (via prescribed burns), the goals of this organization can be met.

<div style="text-align: center;">
    <img src="./pics/map.jpg" alt="Map" style="width: 400px;">
</div>

## Understanding the problem

Forest inventory data consists of the locations, species, heights, and diameters, and more of individual trees in a forested area. 
This valuable data source can help us learn about carbon cycles, timber harvesting, endangered species habitat, and how wildfires will spread.

Traditional forest inventory data is collected manually using a plot-based sampling design. Due to the manual work required, this method of data collection is resource-intensive, giving us a lot of data points for a small area. New techniques in remote sensing and machine learning have opened the doors to new and more efficient ways of collecting and analyzing forest inventory data, but there are still many challenges to overcome. For example, from aerial lidar scans (ALS), we have the ability to quickly obtain heights and locations from trees over a large area (a small amount of data points for a large area).

The goal of this data challenge is to use a massive database of forest inventory data, the FIA database, to predict the size and species composition of trees in a forest from the [Independence Lake Preserve](https://www.nature.org/en-us/get-involved/how-to-help/places-we-protect/independence-lake/) in California. The goals of the challenge are the following:

- Create a model, or set of models, to predict DBH, CBH, and species from tree height using data in the FIA Database- .
- Use the models to make predictions on trees detected from aerial LIDAR at Independence Lake with collected field data for validation# Preliminary analysis.

## Preliminary Analysis
 
The United States Geological Survery (USGS) collects Aerial LIDAR data (ALS) throughout the country, including at the Independence Lake site. 
We can use this point cloud, and a technique called Individual Tree Detection (ITD) to identify individual trees in the forest.

Let's take a look below at the output from the individual tree detection algorithm, tree locations and heights, plotted on a map of the Independence Lake Preserve. 
Click on a dot in the map below to see what we know about the tree at that location.

In [ ]:
# Make sure to have intalled the requirements.txt before start working!
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
DATA_PATH = Path("data")

In [ ]:
# Load the detected trees from the ALS data into a geopandas dataframe
lidar_trees = pd.read_csv(DATA_PATH /'ttops.csv')
lidar_trees = gpd.GeoDataFrame(lidar_trees, geometry=gpd.points_from_xy(lidar_trees.y, lidar_trees.x), crs='EPSG:4326')

# Sample 10,000 trees to speed up the plotting
lidar_trees_sampled = lidar_trees.sample(10000)

In [ ]:
# Add the trees to a folium map.
fmap = folium.Map(location=[lidar_trees_sampled.y.mean(), lidar_trees_sampled.x.mean()], zoom_start=15)
tile = folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='esri').add_to(fmap)
lidar_trees_sampled.apply(lambda p: folium.CircleMarker(
    location=(p["y"], p["x"]),
    radius=3.5, 
    weight=2,
    popup=f"<b>Height:</b> {p.HT:.2f}ft",
).add_to(fmap), axis=1)
fmap

Pretty cool that we can see all this, huh? Did you click on any of the circles representing trees?

You probably saw that besides a tree's location, the tree's height is the only other thing we learn about the tree from the ALS data. But, for many of the problems that we're interested in using the tree inventory data to solve that's not enough data. For example, we're also especially interested in the diameter, the crown base height, and the species of the tree. These are all important variables for estimating how much biomass or carbon is stored in a tree, and, by extension, the forest.  

To get these other variables, we'll want to use what we know, a tree's height and location, as well as additional data, in order to predict the other variables that we're interested in, like diameter, crown base height, and species.

## FIA Database

Fortunately for us, there is an enormous database of tree observations called the FIA database. FIA (Forest Inventory and Analysis) is a program of the US Forest Service that collects data on the nation's forests. The FIA database contains information on the species, height, diameter, and much more for millions of trees across the United States.  

The FIA database is publicly available through the [FIA DataMart](https://apps.fs.usda.gov/fia/datamart/datamart.html), and has [great documentation](https://research.fs.usda.gov/sites/default/files/2023-11/wo-fiadb_user_guide_p2_9-1_final.pdf). The documentation on what data is available in the FIA database is quite extensive, and it's worth taking a look at it to understand what data is available, especially when we start to consider which data we might want to use to predict the variables we're interested in.

The FIA database for the entire inventory area across the United States is around 70GB, consists of around one hundred tables and millions of rows. That's a lot of data! To make this problem more tangible we will begin by looking at just three tables from the database of inventory sites in California. The tables that we will look at are:

1. PLOT Table: Each row contains data about a forest inventory plot such as approximately where it was taken, latitude, and elevation. Plots are identified by a column named "CN".
2. TREE Table: Each row contains data about a tree that was measured. The PLOT_CN column is a foreign key to the CN column in the PLOT table.
3. REF_SPECIES Table: Each row contains data about a tree species, including species code, descriptive common name, scientific name, and many other attributes for each species.

In [ ]:
fia_ca_plot_table = pd.read_csv(DATA_PATH / 'CA_PLOT.csv', low_memory=False)
fia_ca_tree_table = pd.read_csv(DATA_PATH / 'CA_TREE.csv', low_memory=False)
fia_ref_species_table = pd.read_csv(DATA_PATH / 'REF_SPECIES.csv')


### ECOSUBCD

Ecological subsection codes (ECOSUBCD) describe an area of similar surficial geology, lithology, geomorphic
process, soil groups, subregional climate, and potential natural communities. You can read more about how these codes are defined [here](http://www.treesearch.fs.fed.us/pubs/48672).

To make our problem as simple as possible to start, we will only consider observations in the California FIA database that were made in the same ecological subsection code as the Independence Lake Preserve. 
The Independence Lake Preserve has ecological subsection code M261Ej so we filter our initial data on the ECOSUBCD column.

In [ ]:
# Filter the PLOT table for the ECOSUBCD
fia_ca_plot_table = fia_ca_plot_table[fia_ca_plot_table['ECOSUBCD'] == "M261Ej"]

In [ ]:
# Merge the PLOT and TREE tables on the PLT_CN and CN columns
fia_data = pd.merge(fia_ca_tree_table, fia_ca_plot_table, left_on='PLT_CN', right_on='CN')

### Examine the data

In [ ]:
fia_data

As I said, there is an incredible amount of information here.

Let's start by looking at some initial relationships.

In [ ]:
# Plot the relationship between tree height and diameter.
fig, ax = plt.subplots()
fia_data.plot.scatter(x="HT", y="DIA", ax=ax)
ax.set_xlabel("Height (ft)")
ax.set_ylabel("Diameter (in)")
plt.show()

It looks like a tree's height and diameter are related. This is good news for us, because we can use a tree's height to predict its diameter. This is a common relationship in forestry, and there are many models that can be used to predict a tree's diameter from its height.

In [ ]:
# Look at the distribution of tree species
fia_data["SPCD"].value_counts().plot(kind="bar")
plt.show()

It looks like the majority of our tree observations belong to around 6 different species, or species codes (SPCD). But what is a SPCD number? FIA reports species as a field titles SPCD. You can find a list of species codes and their corresponding species names [here](https://www.fs.usda.gov/detailfull/r5/forest-grasslandhealth/?cid=fsbdev3_048105).

Let's make this hitogram again, this time using common names instead of FIA species codes.

In [ ]:
# Create a dictionary mapping SPCD to COMMON_NAME
spcd_to_common_name = dict(zip(fia_ref_species_table['SPCD'], fia_ref_species_table['COMMON_NAME']))
fia_data["COMMON_NAME"] = fia_data["SPCD"].map(spcd_to_common_name)

# Look at the distribution of tree species
fia_data["COMMON_NAME"].value_counts().plot(kind="bar")
plt.show()

In [ ]:
# Compute crown base height from the crown ratio and tree height
fia_data["CBH"] = fia_data["HT"] * (1 - fia_data["CR"] / 100)

# Lastly, lets look at the relationship between crown base height and tree height
fig, ax = plt.subplots()
fia_data.plot.scatter(x="HT", y="CBH", ax=ax)
ax.set_xlabel("Height (ft)")
ax.set_ylabel("Crown Base Height (ft)")
plt.show()


This relationship is quite a bit messier. Maybe there are other variables that we can consider when thinking about this relationship.

# Validation and Evaluation Data

Field crews at the Independent Lake Preserve site have been busy collecting field data with observations on trees throughout the area.
They have made around 300 site specific measurements, and we want to use that data to validate, and ultimately, evaluate the performance of our models. 

In this data challenge we will provide 25% of the full field data to the teams as the validatation dataset. 
We will remove the diameter, species code, and crown base height from the remaining 75% of the observations as the evaluation data.

Teams are responsible for returning the evaluation data with additional DIA (diameter), SPCD (species code), and CBH (crown base height) columns. 
Because FIA uses imperial units, we will expect the added columns to have units of inches for DIA and feet for CBH.

Let's take a look at the valaidation data below.

In [ ]:
# Load the validation data
validation_data = pd.read_csv(DATA_PATH / 'validation_data.csv')

# Rename tree_ht to HT
validation_data.rename(columns={'tree_ht': 'HT'}, inplace=True)

# Convert HT from meters to feet
validation_data["HT"] *= 3.28084

# Rename tree_dbh to DIA
validation_data.rename(columns={'tree_dbh': 'DIA'}, inplace=True)

# Convert DIA from centimeters to inches
validation_data["DIA"] *= 0.393701

# Rename tree_htlcb to CBH
validation_data.rename(columns={'tree_htlcb': 'CBH'}, inplace=True)

# Convert CBH from meters to feet
validation_data["CBH"] *= 3.28084

# Combine the Genus and Species columns in REF_SPECIES and use the tree_sp_scientific_name column to find the SPCD
fia_ref_species_table['SPECIES_NAME'] = fia_ref_species_table['GENUS'] + ' ' + fia_ref_species_table['SPECIES']
validation_data["SPCD"] = validation_data["tree_sp_scientific_name"].apply(lambda x: fia_ref_species_table[fia_ref_species_table['SPECIES_NAME'] == x]['SPCD'].values[0])

# Drop all columns except HT, DIA, CBH, and SPCD
validation_data = validation_data[["HT", "DIA", "CBH", "SPCD"]]

In [ ]:
# Plot the relationship between tree height and diameter.
fig, ax = plt.subplots()
validation_data.plot.scatter(x="HT", y="DIA", ax=ax)
ax.set_xlabel("Height (ft)")
ax.set_ylabel("Diameter (in)")
plt.show()


In [ ]:
# Plot the relationship between tree height and crown base height.
fig, ax = plt.subplots()
validation_data.plot.scatter(x="HT", y="CBH", ax=ax)
ax.set_xlabel("Height (ft)")
ax.set_ylabel("Crown Base Height (ft)")
plt.show()

In [ ]:
# Look at the distribution of tree species
validation_data["SPCD"].value_counts().plot(kind="bar")
plt.show()

What differences do you notice between distributions in the validation data and the FIA data? Is there anything there that can help you tailor a predictive model to the Independence Lake Preserve?

# Next steps

Hopefully you have a good idea of what the data that we're going to be working with throughout the data challenge looks like. 
The next notebook, `model_random_forest.ipynb` walks you through training a simple, univariate random forest models to predict diameter, species code, and crown base height from an observed tree height.
